In [558]:
import math 
  
def msb(x):
    return x.bit_length()


def write(out, value, universe):
    b = msb(universe)
    m =  (int(1) << (b + 1)) - universe
    if value < m:
        if b > 0:
            out += BitVector.BitVector( intVal = value, size = b)
    else:
        value += m;
        if b > 0:
            out += BitVector.BitVector( intVal = (value >> 1), size = b)
        out += BitVector.BitVector( intVal = (value & 1), size = 1 )
    return out 

def read(inp, universe):
    b = msb(universe)
    m = (int(1) << (b+1)) - universe
    val = inp[:b].int_val()
    if inp.length() > b:
        inp = inp[b:]
    else:
        inp =  BitVector.BitVector(size = 0)
    if val >= m:
        val = (val << 1) + inp[:1].int_val() - m
        if inp.length() > 1:
            inp = inp[1:]
        else:
            inp =  BitVector.BitVector(size = 0)
    return val, inp;

def ipc_encode(out, inp, n, low, high):
    if n > 0:
        mid_pos = int(n/2)
        mid_value = inp[mid_pos]
        out = write(out, mid_value - low - mid_pos, high - low - n + 1)
        out = ipc_encode(out, inp[:mid_pos], mid_pos, low, mid_value)
        out = ipc_encode(out, inp[mid_pos+1:], n - mid_pos - 1, mid_value, high)
    return out

def ipc_decode(inp, n, low, high):
    mid_pos = int(n/2)
    v, inp = read(inp, high - low - n + 1)
    value = low + v + mid_pos
    if n == 1:
        return [value], inp
    l = []
    if mid_pos > 0:
        l, inp = ipc_decode(inp, mid_pos, low, value)
    r = []
    if (n - mid_pos - 1) > 0:
        r, inp = ipc_decode(inp, n - mid_pos - 1, value, high)
    return  l + [value] + r, inp


In [583]:
import BitVector

out  = BitVector.BitVector(size = 0)
l = sorted(random.sample(range(0, 100), 99))
out = ipc_encode(out, l, len(l), min(l), max(l))
print("Bits/int: ", out.length()/len(l))
decoded_list = ipc_decode(out, len(l), min(l), max(l))[0]

if decoded_list != l:
    print("Error")
else:
    print("No Error :)")

Bits/int:  1.9090909090909092
No Error :)
